In [ ]:
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display

# חיבור לגוגל דרייב
drive.mount('/content/drive')

# נתיב הקובץ
file_path = "/content/student.txt"  # עדכן את הנתיב לפי המיקום שלך

# קריאה מהקובץ והדפסה
try:
    with open(file_path, "r", encoding="utf-8") as file:
        contents = file.read()
except FileNotFoundError:
    print("⚠️ הקובץ לא נמצא.")

# עיבוד הנתונים ושמירתם במילון
students = {}

for line in contents.strip().split("\n"):
    parts = line.strip().split(",")  # הפרדת הנתונים לפי פסיקים
    if len(parts) >= 2:
        name = parts[0].strip()  # שם הסטודנט
        details = ", ".join(parts[1:]).strip()  # שאר הפרטים
        students[name] = details

# יצירת Dropdown עם שמות הסטודנטים
dropdown = widgets.Dropdown(
    options=["בחר סטודנט"] + list(students.keys()),
    description="סטודנט:",
)

# יצירת תיבת טקסט להצגת הפרטים
output = widgets.Output()

# תיבת טקסט להוספת תוכנית אהובה
favorite_program_text = widgets.Text(
    description="תוכנית אהובה:",
    placeholder="הזן תוכנית אהובה",
)

# כפתור לעדכון תוכנית אהובה
update_button = widgets.Button(description="עדכן תוכנית אהובה")

# פונקציה לעדכון תוכנית אהובה
def update_favorite_program(student_name, favorite_program):
    if student_name in students:
        # החלפת תוכנית אהובה לפרטי הסטודנט
        details = students[student_name]

        # אם קיימת תוכנית אהובה, נחליף אותה
        details_list = details.split(", ")
        # נוודא שהתוכנית האהובה לא קיימת כבר
        details_list = [d for d in details_list if "תוכנית אהובה" not in d]
        details_list.append(f"תוכנית אהובה: {favorite_program}")
        students[student_name] = ", ".join(details_list)

        # כתיבה חזרה לקובץ
        with open(file_path, "w", encoding="utf-8") as file:
            for name, details in students.items():
                file.write(f"{name}, {details}\n")

        print(f"התוכנית {favorite_program} עודכנה בהצלחה לסטודנט {student_name}.")
    else:
        print(f"⚠️ הסטודנט {student_name} לא נמצא.")

# פונקציה שמתבצעת כאשר בוחרים סטודנט
def on_select(change):
    with output:
        output.clear_output()
        if change["new"] in students:
            # הדפסת פרטי הסטודנט
            print(f"פרטי הסטודנט:\n{students[change['new']]}")

            # הצגת תיבת טקסט והכפתור לעדכון תוכנית אהובה
            display(favorite_program_text)
            favorite_program_text.value = ""  # ריקון תיבת הטקסט

            # כפתור לעדכון תוכנית אהובה
            display(update_button)

# פונקציה שמתבצעת כאשר לוחצים על כפתור העדכון
def on_update_button_clicked(b):
    selected_student = dropdown.value
    favorite_program = favorite_program_text.value
    if selected_student != "בחר סטודנט" and favorite_program:
        update_favorite_program(selected_student, favorite_program)

# חיבור בין כפתור העדכון לפונקציה
update_button.on_click(on_update_button_clicked)

# חיבור לצפייה בשינוי הבחירה של הסטודנט
dropdown.observe(on_select, names="value")

# הצגת ה-dropdown ותיבת הפלט
display(dropdown, output)
